In [1]:
from pprint import pprint
from lxml import html
import requests
import datetime,time
import unicodedata
import json
import pandas as pd
from pymongo import MongoClient

In [2]:
yandex_link = 'https://yandex.ru/news/'
mailru_link = 'https://news.mail.ru/'
lenta_link = 'https://lenta.ru/'

In [3]:
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
yandex_request = requests.get(yandex_link,headers=header).text
mailru_request = requests.get(mailru_link,headers=header).text
lenta_request = requests.get(lenta_link,headers=header).text

yandex_html = html.fromstring(yandex_request)
mailru_html = html.fromstring(mailru_request)
lenta_html = html.fromstring(lenta_request)

### YANDEX NEWS

In [4]:
news_link_yandex = yandex_html.xpath('//h2/a[contains(@class, link_theme_black)]/@href')
news_header_yandex = yandex_html.xpath('//h2/a[contains(@class, link_theme_black)]/text()')
news_origin_date_yandex = yandex_html.xpath('//div[@class = "story__date"]/text()')

news_origin_date_yandex1 = news_origin_date_yandex.copy()
for k,i in enumerate(news_link_yandex):
    news_link_yandex[k] = i.replace('/news/',yandex_link)
for k,i in enumerate(news_origin_date_yandex):
    news_origin_date_yandex[k] = unicodedata.normalize('NFKD',i).split(' ')
    news_origin_date_yandex[k][0] = ' '.join(news_origin_date_yandex[k][:-1])
    if str('вчера в') in news_origin_date_yandex[k][0]:
        news_origin_date_yandex[k][0]=news_origin_date_yandex[k][0].replace('вчера в','')
        news_origin_date_yandex[k][-1] = 'вчера в ' + news_origin_date_yandex[k][-1]

for z,k in enumerate([i[-1] for i in news_origin_date_yandex]):
    if len(k)==5:
        news_origin_date_yandex[z][-1] = str(datetime.date.today())
    elif 'вчера' in str.lower(k):
        news_origin_date_yandex[z][-1] = str(datetime.date.today()-datetime.timedelta(days=1))

for i in zip(news_header_yandex,news_link_yandex,[i[0] for i in news_origin_date_yandex],[i[-1] for i in news_origin_date_yandex]):
    #print (i)
    pass

### LENTA NEWS

In [5]:
news_link_lenta = lenta_html.xpath('//div[@class = "item"]//a/@href|//div[@class = "item news b-tabloid__topic_news"]//h3/a/@href')
news_header_lenta = lenta_html.xpath('//div[@class = "item"]//a/text()|//div[@class = "item news b-tabloid__topic_news"]//h3/a/span/text()')

for k,i in enumerate(news_header_lenta):
    news_header_lenta[k]=unicodedata.normalize('NFKD',i)
news_date_lenta = []
for i in news_link_lenta:
    if len(i.split('.htm'))>1:
        news_date_lenta.append('https://lenta.ru' + '/'.join(''.join(list(i.split('.htm')[0])[-10:]).split('-')[::-1]))
    else:
        news_date_lenta.append('/'.join(i.split('/')[2:5]))
news_origin_lenta = []
for k,i in enumerate(news_link_lenta):
    if str(i)[0] == '/':
        news_origin_lenta.append('Lenta.Ru')
    else:
        news_origin_lenta.append(i.replace('https://','').split('/')[0].title())

for i in zip(news_header_lenta,news_link_lenta,news_origin_lenta,news_date_lenta):
    #print (i)
    pass

### MAIL.RU NEWS

In [6]:
news_link_mailru = mailru_html.xpath('//div[contains(@class,"daynews__item")]//a/@href|//a[@class="list__text"]/@href|//a[@class="link link_flex"]/@href|//a[@class="newsitem__title link-holder"]/@href')
news_header_mailru = mailru_html.xpath('//div[contains(@class,"daynews__item")]//a//span[contains(@class,"photo__title_new")]/text()|//a[@class="list__text"]/text()|//a[@class="link link_flex"]/span/text()|//a[@class="newsitem__title link-holder"]/span/text()')

for k,i in enumerate(news_header_mailru):
    news_header_mailru[k]=unicodedata.normalize('NFKD',i)
for k,i in enumerate(news_link_mailru):
    if str(i)[0] == '/':
        news_link_mailru[k] = mailru_link + str(i)[1:]
    else:
        continue
# если - вдруг - источник не найдется (или не указан, как в lenta.ru), примем, что источником является mail.ru
news_origin_mailru = []
for i in news_link_mailru:
    news_origin_mailru.append(i.replace('https://','').split('/')[0].title())

news_date_mailru = []
for k,i in enumerate(news_link_mailru):
    time.sleep(1)
    news_date_request = html.fromstring(requests.get(i,headers = header).text)
    news_date = news_date_request.xpath('//span[contains(@class,"note__text breadcrumbs__text js-ago")]/@datetime')
    try:
        news_origin_mailru[k]=news_date_request.xpath('//span[@class="breadcrumbs__item"]//span[@class="link__text"]/text()')[0]
    except:
        pass
    if news_date:
        news_date_mailru.append(str(news_date[0]).split('T')[0])
    else:
        news_date = news_date_request.xpath('//span[contains(@class,"breadcrumbs__item js-ago")]/@datetime')
        if news_date:
            news_date_mailru.append(str(news_date[0]).split('T')[0])
        else:
            news_date = news_date_request.xpath('//time[contains(@class,"breadcrumbs__text js-ago")]/@datetime')
            if news_date:
                news_date_mailru.append(str(news_date[0]).split('T')[0])
            else:
                news_date = json.loads(news_date_request.xpath('//script[@type="application/ld+json"]/text()')[0])['datePublished']
                news_date_mailru.append(news_date)

for i in zip(news_header_mailru,news_link_mailru,news_origin_mailru,news_date_mailru):
    #print (i)
    pass

In [7]:
client = MongoClient('localhost', 27017)

db = client['db_of_news']

yandex_db = db.yandex_news
mailru_db = db.mailru
lenta_db = db.lenta

In [8]:
news_to_add_mailru = {}
k=0
for i in news_header_mailru:
    news_to_add_mailru[k]={'title':news_header_mailru[k],'urls':news_link_mailru[k],'source':news_origin_mailru[k],'date':news_date_mailru[k]}
    k+=1
k=0
news_to_add_lenta = {}
for i in news_header_lenta:
    news_to_add_lenta[k]={'title':news_header_lenta[k],'urls':news_link_lenta[k],'source':news_origin_lenta[k],'date':news_date_lenta[k]}
    k+=1
k=0
news_to_add_yandex = {}
for i in news_header_yandex:
    news_to_add_yandex[k]={'title':news_header_yandex[k],'urls':news_link_yandex[k],'source':[i[0] for i in news_origin_date_yandex][k],'date':[i[-1] for i in news_origin_date_yandex][k]}
    k+=1

In [9]:
for i in news_to_add_mailru:
    mailru_db.insert_one(news_to_add_mailru[i])
for i in news_to_add_lenta:
    lenta_db.insert_one(news_to_add_lenta[i])
for i in news_to_add_yandex:
    yandex_db.insert_one(news_to_add_yandex[i])

In [10]:
# можно наблюдать, что некоторые новости из разных блоков собраны из mail.ru дважды
pd.DataFrame(mailru_db.find())['urls'].value_counts()

https://news.mail.ru/politics/41004360/                                                                                                 3
https://news.mail.ru/incident/41004205/                                                                                                 3
https://health.mail.ru/news/immunitet_poka_ne_raspoznaet_koronavirus_glava/                                                             2
https://news.mail.ru/politics/41002334/                                                                                                 2
https://news.mail.ru/economics/41004238/                                                                                                2
https://news.mail.ru/economics/41003305/                                                                                                2
https://kino.mail.ru/news/52884_alena_hmelnitskaya_soobschila_chto_3_5_mesyatsa_bila_bez_raboti/                                        2
https://news.mail.ru/incident/4098

In [11]:
pd.DataFrame(yandex_db.find())['urls'].value_counts()

https://yandex.ru/news/story/Zamglavy_MCHS_Tatarstana_arestovan_po_podozreniyu_v_moshennichestve--0817c0148958d8b0accdc4e0d0f13090?lr=2&lang=ru&stid=PQdFwYndL1sd6ncgnm7n&persistent_id=90892839&rubric=incident&from=index          3
https://yandex.ru/news/story/Peterburg_i_Lenoblast_obedinyayut_usiliya_v_borbe_s_koronavirusom--8b1ea530e54a9d8324bf4d3fbc310fc5?lr=2&lang=ru&stid=Mwm5-m-EbpBAmnr1MpnJ&persistent_id=90900304&rubric=Saint_Petersburg&from=index    3
https://yandex.ru/news/story/V_Rossii_poyavitsya_novyj_avtomobilnyj_dokument--8aa544364a09c1e7148aa75435c010a5?lr=2&lang=ru&stid=2oph-NUGZv1M2raxz0Eq&persistent_id=90491129&rubric=auto&from=index                                  3
https://yandex.ru/news/story/S_plakatov_o_golosovanii_po_Konstitucii_ubrali_datu_22_aprelya--ec174d0dae1193281d6cf4e0b3a61426?lr=2&lang=ru&stid=qA0EehyUeUUlG4ww1PLG&persistent_id=90983973&rubric=politics&from=index               3
https://yandex.ru/news/story/Apple_anonsirovala_novye_iPad_Pro_i_MacBook_Air

In [12]:
pd.DataFrame(lenta_db.find())['urls'].value_counts()

/news/2020/03/18/poezd/          3
/news/2020/03/18/coronastras/    3
/news/2020/03/18/kadyroff/       3
/news/2020/03/18/recession/      3
/news/2020/03/18/croc/           3
                                ..
/news/2020/03/18/alina/          3
/news/2020/03/18/30_msk/         3
/news/2020/03/18/lingerie/       3
/news/2020/03/18/belarus/        3
/news/2020/03/19/erdogan/        1
Name: urls, Length: 70, dtype: int64

In [13]:
# создаем датафрейм и пандовскую серию для отбора urls, которые можно удалить из БД
df = pd.DataFrame(pd.DataFrame(mailru_db.find())['urls'].value_counts())
df.reset_index(inplace=True)
list_to_delete = []
for i in df.loc[df['urls']>1,'index']:
    k = int(df.loc[df['index']==i,'urls'])
    while k>1:
        list_to_delete.append(i)
        k-=1
for i in list_to_delete:
    mailru_db.delete_one({'urls':i})
    
df = pd.DataFrame(pd.DataFrame(yandex_db.find())['urls'].value_counts())
df.reset_index(inplace=True)
list_to_delete = []
for i in df.loc[df['urls']>1,'index']:
    k = int(df.loc[df['index']==i,'urls'])
    while k>1:
        list_to_delete.append(i)
        k-=1
for i in list_to_delete:
    yandex_db.delete_one({'urls':i})
    
df = pd.DataFrame(pd.DataFrame(lenta_db.find())['urls'].value_counts())
df.reset_index(inplace=True)
list_to_delete = []
for i in df.loc[df['urls']>1,'index']:
    k = int(df.loc[df['index']==i,'urls'])
    while k>1:
        list_to_delete.append(i)
        k-=1
for i in list_to_delete:
    lenta_db.delete_one({'urls':i})

In [14]:
# смотрим результат
pd.DataFrame(mailru_db.find())['urls'].value_counts()

https://health.mail.ru/news/immunitet_poka_ne_raspoznaet_koronavirus_glava/                                                             1
https://news.mail.ru/politics/41002334/                                                                                                 1
https://news.mail.ru/economics/41002725/                                                                                                1
https://news.mail.ru/economics/41004238/                                                                                                1
https://news.mail.ru/economics/41003305/                                                                                                1
https://kino.mail.ru/news/52884_alena_hmelnitskaya_soobschila_chto_3_5_mesyatsa_bila_bez_raboti/                                        1
https://news.mail.ru/incident/40988143/                                                                                                 1
https://news.mail.ru/economics/410

In [15]:
pd.DataFrame(yandex_db.find())['urls'].value_counts()

https://yandex.ru/news/story/Zamglavy_MCHS_Tatarstana_arestovan_po_podozreniyu_v_moshennichestve--0817c0148958d8b0accdc4e0d0f13090?lr=2&lang=ru&stid=PQdFwYndL1sd6ncgnm7n&persistent_id=90892839&rubric=incident&from=index    1
https://yandex.ru/news/story/Mishustin_poruchil_podgotovit_zakonoproekt_o_moratorii_na_bankrotstvo--799de18573da1d84586020f2b9731961?lr=2&lang=ru&stid=rcuhDbl8ZAffoMOScjmv&persistent_id=90954824&rubric=index&from=index     1
https://yandex.ru/news/story/Minzdrav_Ukrainy_podtverdil_zarazhenie_koronavirusom_deputata_Rady--96d06b3dd09b482ba5b1ffef911a973f?lr=2&lang=ru&stid=fXDkjkPZcZG_&persistent_id=90958177&rubric=personal_feed&from=index        1
https://yandex.ru/news/story/V_aehroportu_Belgoroda_otkryvayutsya_polety_v_Peterburg--7fae44b79ea6c0172483bb6347777cf9?lr=2&lang=ru&stid=oC2UDwT1spz1rrj59moO&persistent_id=90963721&rubric=Saint_Petersburg&from=index        1
https://yandex.ru/news/story/V_FAS_prokommentirovali_rost_cen_na_benzin--46230f1f5cdba052cfb288a2a4c

In [16]:
pd.DataFrame(lenta_db.find())['urls'].value_counts()

/news/2020/03/18/iran/                     1
/news/2020/03/18/kritikachistogorazuma/    1
/news/2020/03/18/kapital/                  1
/news/2020/03/18/kadyroff/                 1
/news/2020/03/18/recession/                1
                                          ..
/news/2020/03/18/30_msk/                   1
/news/2020/03/18/lingerie/                 1
/news/2020/03/18/visa/                     1
/news/2020/03/18/frcor/                    1
/news/2020/03/18/belarus/                  1
Name: urls, Length: 70, dtype: int64